In [1]:
#importing libraries
import spacy
import pandas as pd
import nltk
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier

In [2]:
#loading the spacy pretrained model on English news
nlp = spacy.load('en_core_web_md')

In [10]:
data = pd.read_csv('cleantrainingdata.csv')

In [16]:
#preparing the training input and output

xtrain=[]
ytrain = []


l = data['length']
sim = data['similarity']
neg = data['neg_score']
neut = data['neut_score']
pos = data['pos_score']
comp = data['comp_score']
hed = data['count_hedgewords']
post=data['text']
ide=data['thread_id']
label=data['delta']
auth = data['author']
auth_score = data['author_score']
auth_flair = data['author_flair']


for i in range(1, ide.size):
    #v = [sim.loc[i],neg.loc[i],pos.loc[i],comp.loc[i],hed.loc[i],auth_score.loc[i],auth_flair[i]]
    v = [l.loc[i],sim.loc[i],comp.loc[i],hed.loc[i],auth_score.loc[i],auth_flair[i]]
    lab = label.loc[i]
    xtrain.append(v)
    ytrain.append(lab)

In [18]:
#random forrest classifier 
xtrain=np.array(xtrain).astype(np.int32)
ytrain=np.array(ytrain).astype(np.int32)

rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)
#print('Random Forest percentage score on 25% validation=',rfc.score(xtrain,ytrain)*100)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
datat = pd.read_csv('cleantestingdata.csv')

In [20]:
#preparing test input and output
xtest=[]
ytest = []

lt = datat['length']
simt = datat['similarity']
negt = datat['neg_score']
neutt = datat['neut_score']
post = datat['pos_score']
compt = datat['comp_score']
hedt = datat['count_hedgewords']
postt=datat['text']
idet=datat['thread_id']
labelt=datat['delta']
autht = datat['author']
auth_scoret = datat['author_score']
auth_flairt = datat['author_flair']

for i in range(1, idet.size):
    vt = [lt.loc[i],simt.loc[i],compt.loc[i],hedt.loc[i],auth_scoret.loc[i],auth_flairt.loc[i]]
    labt = labelt.loc[i]
    xtest.append(vt)
    ytest.append(labt)

In [21]:
ypred = rfc.predict(xtest)
ypred_list = ypred.tolist()

#print(rfc.score(xtest,ytest)*100)


#Save ypred in csv file
dataypred = pd.DataFrame(columns=['predicted_delta'])
dataypred['predicted_delta']=ypred_list

dataypred.to_csv('predicteddelta.csv')

In [22]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


print('\n Confusion Matrix: \n', confusion_matrix(ytest,ypred,labels=[0,1]))
print('\n Accuracy: ', accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall, F1-score: \n',classification_report(ytest,ypred))
s = cohen_kappa_score(ytest,ypred)
print('\n Cohen Kappa Score: ',s)


 Confusion Matrix: 
 [[231  69]
 [ 76 222]]

 Accuracy:  75.75250836120402

 Precision, Recall, F1-score: 
               precision    recall  f1-score   support

           0       0.75      0.77      0.76       300
           1       0.76      0.74      0.75       298

   micro avg       0.76      0.76      0.76       598
   macro avg       0.76      0.76      0.76       598
weighted avg       0.76      0.76      0.76       598


 Cohen Kappa Score:  0.5150067678677301
